# Generate image(original)

In [ ]:
# %load generate.py
# speed up the loading of the training data
%load_ext autoreload
%autoreload 2
import cv2
import os
import torch as th
from model import NetG, NetD, NetA
from data_set import CASIABDatasetGenerate
import visdom
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
# checkpoint = './Model_64x64_allseqs_pixelDTcode/snapshotModel_64x64_allseqs_pixelDTcode_30000.t7'
checkpoint = './Model_64x64_allseqs_newdata/snapshotModel_64x64_allseqs_newdata_25000.t7'

# vis = visdom.Visdom(port=5274)
# win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")
netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
fineSize = 64

# checkpoint = './snapshot64_20000.t7'
checkpoint = th.load(checkpoint)
neta.load_state_dict(checkpoint['netA'])
netg.load_state_dict(checkpoint['netG'])
netd.load_state_dict(checkpoint['netD'])
neta.eval()
netg.eval()
netd.eval()
angles = ['000', '018', '036', '054', '072', '090',
          '108', '126', '144', '162', '180']
if not os.path.isdir('./Transform'+Data_Name):
    os.mkdir('./Transform'+Data_Name)
    
for cond in ['nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05',
             'nm-06']:
    dataset = CASIABDatasetGenerate(data_dir=Data_Dir,cond=cond)
    print('cond',cond)
    for i in range(1, 125):
        ass_label, img = dataset.getbatch(i, 11)
#         img = dataset.getbatch(i, 10)
        img = img.to(device).to(th.float32)

        with th.no_grad():
            fake = netg(img)
            fake = (fake + 1) / 2 * 255
            for j in range(11):
                fake_ = fake[j].squeeze().cpu().numpy()
                ang = angles[j]
                cv2.imwrite('./Transform'+Data_Name+'/%03d-%s-%s.png' % (i, cond, ang), fake_)
        
        


# Generate image

In [1]:
# %load generate.py
# speed up the loading of the training data
import cv2
import os
import torch as th
# from model import NetG, NetD, NetA
# from model_siGAN_dropout import NetG, NetD, NetA
# from model_WGAN import NetG, NetD, NetA
# from model_SAGAN1_1 import NetG, NetD, NetA
# from model_SAGAN import NetG, NetD, NetA
from model import NetG, NetD, NetA
from dataset2Loader import loadImage
import visdom

checkpoint = './Transform_Model/Model_64x64_GaitGAN_90/snapshotModel_64x64_GaitGAN_90_600.t7'
# checkpoint = './Transform_Model/Model_64x64_TripletSAGAN_90_trial3/snapshotModel_64x64_TripletSAGAN_90_trial3_600.t7'
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
save_datadir = './Transform_64x64_gaitgan'

netg = NetG(nc=1)
device = th.device("cuda:1")
netg = netg.to(device)

checkpoint = th.load(checkpoint)
netg.load_state_dict(checkpoint['netG'])
netg.eval()
angles = ['000', '018', '036', '054', '072', '090',
          '108', '126', '144', '162', '180']
if not os.path.isdir(save_datadir):
    os.mkdir(save_datadir)


for cond in ['nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05','nm-06','bg-01','bg-02','cl-01','cl-02']:
    for i in range(63, 125): 
        id1 = '%03d' % i
        for ang in angles:
            r3 = id1 + '/' + cond + '/'  + id1 + '-' + cond + '-' + ang + '.png'
            if os.path.exists(Data_Dir + r3):
                img3 = loadImage(Data_Dir + r3) #(1,64,64) 且歸一化
                img3 = img3.unsqueeze(0)  #(1,1,64,64)
                img3 = img3.to(device).to(th.float32)
                with th.no_grad():
#                     fake,_ = netg(img3)
                    fake = netg(img3)
                    fake = (fake + 1) / 2 * 255
                    fake_ = fake.squeeze().cpu().numpy()
                    cv2.imwrite(save_datadir+'/%03d-%s-%s.png' % (i, cond, ang), fake_)
            else:
                print('None', r3)
                continue

                    



None 068/nm-01/068-nm-01-054.png
None 068/nm-01/068-nm-01-126.png
None 068/nm-01/068-nm-01-144.png
None 068/nm-01/068-nm-01-162.png
None 109/nm-01/109-nm-01-126.png
None 109/nm-01/109-nm-01-144.png
None 109/nm-01/109-nm-01-162.png
None 109/nm-01/109-nm-01-180.png
None 068/nm-02/068-nm-02-126.png
None 068/nm-02/068-nm-02-162.png
None 068/nm-03/068-nm-03-126.png
None 068/nm-03/068-nm-03-144.png
None 068/nm-03/068-nm-03-180.png
None 088/nm-03/088-nm-03-054.png
None 088/nm-03/088-nm-03-126.png
None 088/nm-03/088-nm-03-162.png
None 088/nm-03/088-nm-03-180.png
None 068/nm-04/068-nm-04-126.png
None 068/nm-04/068-nm-04-144.png
None 088/nm-04/088-nm-04-054.png
None 088/nm-04/088-nm-04-126.png
None 088/nm-04/088-nm-04-144.png
None 088/nm-04/088-nm-04-162.png
None 088/nm-04/088-nm-04-180.png
None 079/bg-02/079-bg-02-054.png
None 079/bg-02/079-bg-02-162.png


# Generate image by multi-view

In [1]:
# %load generate.py
# speed up the loading of the training data
import cv2
import os
import torch as th
# from model_dropout import NetG, NetD, NetA
# from model_siGAN_dropout import NetG, NetD, NetA
# from model_WGAN import NetG, NetD, NetA
from model_siGAN import NetG, NetD, NetA
# from model import NetG, NetD, NetA
from dataset2Loader import loadImage
import visdom

checkpoint36 = './Transform_Model/Model_64x64_TripletGAN_36_trial1/lowest_snapshotModel_64x64_TripletGAN_36_trial1_598.t7'
checkpoint90 = './Transform_Model/Model_64x64_SAGAN_90_trial1/lowest_snapshotModel_64x64_TripletGAN_90_trial1_613.t7'
checkpoint144 = './Transform_Model/Model_64x64_TripletGAN_144_trial1/lowest_snapshotModel_64x64_TripletGAN_144_trial1_613.t7'
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
save_datadir = './Transform_64x64_allseq'

netg = NetG(nc=1)
device = th.device("cuda:1")
netg = netg.to(device)

def choice_model(checkpoint,netG):
    checkpoint = th.load(checkpoint)
    netG.load_state_dict(checkpoint['netG'])
    netG.eval()
    return netG

angles = ['000', '018', '036', '054', '072', '090',
          '108', '126', '144', '162', '180']
if not os.path.isdir(save_datadir):
    os.mkdir(save_datadir)

count = 0
for cond in ['nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05','nm-06','bg-01','bg-02','cl-01','cl-02']:
    for i in range(63, 125): 
        id1 = '%03d' % i
        for ang in angles: 
            r3 = id1 + '/' + cond + '/'  + id1 + '-' + cond + '-' + ang + '.png'
            if os.path.exists(Data_Dir + r3):
                img3 = loadImage(Data_Dir + r3) 
                img3 = img3.unsqueeze(0)  #(1,1,64,64)
                img3 = img3.to(device).to(th.float32)
                if ang in ['000', '018', '036']:  
#                     print(r3,'in 1 ang')
                    with th.no_grad():
                        netg= choice_model(checkpoint36,netg)
                        fake36,_ = netg(img3)
                        netg= choice_model(checkpoint90,netg)
#                         fake36 = (fake36 + 1) / 2 * 255
                        fake90,_ = netg(fake36)
                elif ang in ['054', '072', '090', '108', '126']:
#                     print(r3,'in 2 ang')
                    with th.no_grad():
                        netg= choice_model(checkpoint90,netg)
                        fake90,_ = netg(img3)
                elif ang in ['144', '162', '180']:
#                     print(r3,'in 3 ang')
                    with th.no_grad():
                        netg= choice_model(checkpoint144,netg)
                        fake144,_ = netg(img3)
                        netg= choice_model(checkpoint90,netg)
#                         fake144 = (fake144 + 1) / 2 * 255
                        fake90,_ = netg(fake144)
                fake90 = (fake90 + 1) / 2 * 255
                fake90 = fake90.squeeze().cpu().numpy()
                cv2.imwrite(save_datadir+'/%03d-%s-%s.png' % (i, cond, ang), fake90)

                    



KeyboardInterrupt: 

# show generated images of some people on vistom

In [4]:
%load_ext autoreload
%autoreload 2
import cv2
import torch as th
from model_SAGAN import NetG, NetD, NetA
from data_set import CASIABDatasetGenerate,loadImage
import visdom
checkpoint = './Transform_Model/Model_64x64_SAGAN_90_trial1/lowest_snapshotModel_64x64_SAGAN_90_trial1_697.t7'
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
from torchvision.utils import make_grid


vis = visdom.Visdom(port=8097)
win2 = None
win3 = None
win5 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
# neta = NetA(nc=1)
device = th.device("cuda:1")
netg = netg.to(device)
netd = netd.to(device)
# neta = neta.to(device)
fineSize = 64

checkpoint = th.load(checkpoint)
# neta.load_state_dict(checkpoint['netA'])
netg.load_state_dict(checkpoint['netG'])
netd.load_state_dict(checkpoint['netD'])
# neta.eval()
netg.eval()
netd.eval()
angles = ['000', '018', '036', '054', '072', '090',
          '108', '126', '144', '162', '180']
for cond in [
#     'nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05',
             'nm-06']:
    dataset = CASIABDatasetGenerate(data_dir=Data_Dir ,cond=cond)
    print('cond',cond)
    for i in range(100, 105):
        ass_label, img = dataset.getbatch(i, 11)
#         img = dataset.getbatch(i, 11)
#         print(img.shape)
        ass_label = ass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        with th.no_grad():
            fake,_= netg(img)
#             fake = (fake + 1) / 2 * 255
#             for j in range(11):
#                 fake_ = fake[j].squeeze().cpu().numpy()
#                 ang = angles[j]
#                 cv2.imwrite('./Transform64/%03d-%s-%s.png' % (i, cond, ang), fake_)
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 11).cpu().numpy()
        
        if win5 is None:
            win5 = vis.image(display,
                             opts=dict(title="train", caption='train')) #最後會印出最後一個狀態 NM06
        else:
            vis.image(display, win=win5)
        win5 = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cond nm-06


In [ ]:
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline 
colors = ['red','green','blue','purple','yellow','black','brown','pink','orange','gray']
plt.scatter(X_embedded[:1000,0],X_embedded[:1000,1], c=label, cmap=matplotlib.colors.ListedColormap(colors))
cb = plt.colorbar()
loc = np.arange(0,max(label),max(label)/float(len(colors)))
cb.set_ticks(loc)
cb.set_ticklabels([a for a in range(10)])
plt.show()